In [ ]:
"""
Personal Finance Tracker
File: personal_finance_tracker.py

Features:
- Add income/expense transactions (auto timestamped)
- View all transactions (with optional filters)
- Summary: total income, total expense, balance
- Category-wise expense visualization (bar chart, if matplotlib installed)
- Search transactions
- Delete transaction by ID
- Backup CSV copy
- Export to SQLite (optional, if sqlite3 available)

CSV file used: finance_data.csv
Columns: id,date,type,amount,category,description
"""

import csv
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import uuid
from datetime import datetime
from shutil import copyfile

HAS_PANDAS = False
HAS_MATPLOTLIB = False
try:
    import pandas as pd
    HAS_PANDAS = True
except Exception:
    HAS_PANDAS = False

try:
    import matplotlib.pyplot as plt
    HAS_MATPLOTLIB = True
except Exception:
    HAS_MATPLOTLIB = False

CSV_FILE = "finance_data.csv"
CSV_FIELDS = ["id", "date", "type", "amount", "category", "description"]


def ensure_csv_exists():
    """Create CSV file with header if not present."""
    if not os.path.exists(CSV_FILE):
        with open(CSV_FILE, mode="w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=CSV_FIELDS)
            writer.writeheader()


def read_all_transactions():
    """Read CSV and return list of dict rows."""
    ensure_csv_exists()
    rows = []
    with open(CSV_FILE, mode="r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for r in reader:

            r["amount"] = float(r["amount"]) if r["amount"] != "" else 0.0
            rows.append(r)
    return rows


def write_all_transactions(rows):
    """Overwrite CSV with provided list of dict rows."""
    with open(CSV_FILE, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=CSV_FIELDS)
        writer.writeheader()
        for r in rows:

            out = {
                "id": r["id"],
                "date": r["date"],
                "type": r["type"],
                "amount": str(r["amount"]),
                "category": r["category"],
                "description": r["description"],
            }
            writer.writerow(out)


def add_transaction():
    """Prompt user and add a transaction to CSV."""
    print("\n--- Add Transaction ---")
    ttype = ""
    while ttype not in ("Income", "Expense"):
        ttype_in = input("Type (Income/Expense): ").strip().title()
        if ttype_in in ("Income", "Expense"):
            ttype = ttype_in
        else:
            print("Please enter 'Income' or 'Expense'.")

    while True:
        amt_in = input("Amount (numbers only, e.g., 2500.50): ").strip()
        try:
            amount = float(amt_in)
            break
        except ValueError:
            print("Invalid amount. Try again.")

    category = input("Category (e.g., Food, Travel, Salary): ").strip() or "General"
    description = input("Short description (optional): ").strip()

    trans_id = uuid.uuid4().hex[:8]
    date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    new_row = {
        "id": trans_id,
        "date": date,
        "type": ttype,
        "amount": amount,
        "category": category,
        "description": description,
    }

    ensure_csv_exists()
    with open(CSV_FILE, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=CSV_FIELDS)
        writer.writerow({
            "id": new_row["id"],
            "date": new_row["date"],
            "type": new_row["type"],
            "amount": str(new_row["amount"]),
            "category": new_row["category"],
            "description": new_row["description"],
        })

    print(f"Transaction added with ID: {trans_id}")


def view_transactions(limit=0):
    """Display transactions. limit=0 => all"""
    rows = read_all_transactions()
    if not rows:
        print("\nNo transactions found.\n")
        return

    print("\n--- Transactions ---")
    header = f"{'ID':8}  {'Date':19}  {'Type':8}  {'Amount':10}  {'Category':15}  Description"
    print(header)
    print("-" * len(header))
    count = 0
    for r in rows:
        print(f"{r['id']:8}  {r['date'][:19]:19}  {r['type']:8}  {r['amount']:10.2f}  {r['category'][:15]:15}  {r['description']}")
        count += 1
        if limit and count >= limit:
            break
    print(f"\nTotal records: {len(rows)}\n")


def view_summary():
    """Compute and display totals and balance."""
    rows = read_all_transactions()
    total_income = sum(r["amount"] for r in rows if r["type"] == "Income")
    total_expense = sum(r["amount"] for r in rows if r["type"] == "Expense")
    balance = total_income - total_expense

    print("\n--- Summary ---")
    print(f"Total Income : ₹{total_income:,.2f}")
    print(f"Total Expense: ₹{total_expense:,.2f}")
    print(f"Balance      : ₹{balance:,.2f}\n")


def visualize_expenses():
    """Plot category-wise expenses (bar chart)."""
    rows = read_all_transactions()
    expenses = [r for r in rows if r["type"] == "Expense"]
    if not expenses:
        print("No expenses to plot.")
        return

    cat_totals = {}
    for e in expenses:
        cat = e["category"] or "Uncategorized"
        cat_totals[cat] = cat_totals.get(cat, 0.0) + float(e["amount"])

    if not HAS_MATPLOTLIB:
        print("\nmatplotlib not installed. Install it to see charts: pip install matplotlib")
        print("Category-wise totals (text):")
        for k, v in cat_totals.items():
            print(f"{k}: ₹{v:,.2f}")
        return

    categories = list(cat_totals.keys())
    values = [cat_totals[c] for c in categories]

    plt.figure(figsize=(8, 5))
    plt.bar(categories, values)
    plt.xlabel("Category")
    plt.ylabel("Amount (₹)")
    plt.title("Category-wise Expenses")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()


def search_transactions():
    """Search by term in category/description or by date/id."""
    rows = read_all_transactions()
    if not rows:
        print("No transactions available to search.")
        return

    term = input("Enter search term (category / description / id / date YYYY-MM-DD): ").strip().lower()
    results = []
    for r in rows:
        if term in r["id"].lower() or term in r["category"].lower() or term in r["description"].lower() or term in r["date"].lower():
            results.append(r)

    if not results:
        print("No matching transactions found.")
        return

    print(f"\nFound {len(results)} matching records:")
    for r in results:
        print(f"{r['id']:8}  {r['date'][:19]:19}  {r['type']:8}  {r['amount']:10.2f}  {r['category'][:15]:15}  {r['description']}")


def delete_transaction():
    """Delete a transaction by ID."""
    rows = read_all_transactions()
    if not rows:
        print("No transactions to delete.")
        return

    tid = input("Enter transaction ID to delete: ").strip()
    new_rows = [r for r in rows if r["id"] != tid]
    if len(new_rows) == len(rows):
        print("Transaction ID not found.")
        return

    write_all_transactions(new_rows)
    print(f"Transaction {tid} deleted.")


def backup_csv():
    """Create a timestamped backup of the CSV file."""
    ensure_csv_exists()
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_name = f"finance_data_backup_{now}.csv"
    copyfile(CSV_FILE, backup_name)
    print(f"Backup created: {backup_name}")


def export_to_sqlite():
    """Optional: export CSV to a local SQLite DB (if sqlite3 available)."""
    try:
        import sqlite3
    except Exception:
        print("sqlite3 not available in this Python environment.")
        return

    db_name = "finance_data.db"
    conn = sqlite3.connect(db_name)
    c = conn.cursor()
    c.execute("""
        CREATE TABLE IF NOT EXISTS transactions (
            id TEXT PRIMARY KEY,
            date TEXT,
            type TEXT,
            amount REAL,
            category TEXT,
            description TEXT
        )
    """)
    rows = read_all_transactions()
    for r in rows:
        c.execute("""
            INSERT OR REPLACE INTO transactions (id, date, type, amount, category, description)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (r["id"], r["date"], r["type"], r["amount"], r["category"], r["description"]))
    conn.commit()
    conn.close()
    print(f"Exported {len(rows)} records to {db_name}")


def clear_screen():
    if os.name == 'nt':
        os.system('cls')
    else:
        os.system('clear')


def main_menu():
    ensure_csv_exists()
    while True:
        print("\n=== Personal Finance Tracker ===")
        print("1. Add transaction")
        print("2. View transactions (all)")
        print("3. View summary (income, expense, balance)")
        print("4. Visualize expenses (category-wise)")
        print("5. Search transactions")
        print("6. Delete transaction by ID")
        print("7. Backup CSV")
        print("8. Export to SQLite (optional)")
        print("9. Exit")
        choice = input("Choose an option (1-9): ").strip()
        if choice == "1":
            add_transaction()
        elif choice == "2":
            view_transactions()
        elif choice == "3":
            view_summary()
        elif choice == "4":
            visualize_expenses()
        elif choice == "5":
            search_transactions()
        elif choice == "6":
            delete_transaction()
        elif choice == "7":
            backup_csv()
        elif choice == "8":
            export_to_sqlite()
        elif choice == "9":
            print("Goodbye — data saved in", CSV_FILE)
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 9.")


if __name__ == "__main__":
    try:
        main_menu()
    except KeyboardInterrupt:
        print("\nInterrupted. Exiting.")
        sys.exit(0)


=== Personal Finance Tracker ===
1. Add transaction
2. View transactions (all)
3. View summary (income, expense, balance)
4. Visualize expenses (category-wise)
5. Search transactions
6. Delete transaction by ID
7. Backup CSV
8. Export to SQLite (optional)
9. Exit

--- Add Transaction ---
Transaction added with ID: 4e7e9158

=== Personal Finance Tracker ===
1. Add transaction
2. View transactions (all)
3. View summary (income, expense, balance)
4. Visualize expenses (category-wise)
5. Search transactions
6. Delete transaction by ID
7. Backup CSV
8. Export to SQLite (optional)
9. Exit

--- Transactions ---
ID        Date                 Type      Amount      Category         Description
---------------------------------------------------------------------------------
b895fc1c  2025-11-06 13:07:42  Expense     40000.00  Travel           HI
4e7e9158  2025-11-06 13:18:01  Expense     80000.00  Salary           err

Total records: 2


=== Personal Finance Tracker ===
1. Add transaction
2. V